## Policing in Schools
A look at whether school policing can predict graduation rates.

By Onel Abreu, Alexander Roche, Sabrina Sedovic

In [ ]:
import pandas as pd
import pipeline as pipeline

school_data = pd.read_csv("school_data.csv")